In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#matplotlib inline

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

import sys
sys.path.append('./src')

from data_queries import  getClusterInfo, getGAIAKnownMembers, querySIMBAD, formatSIMBADtoGAIA
from gaiastars import gaiastars as gs

import astropy.units as u

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
#construct a dict mapping cluster name in Table1a to its name in Simabad
name_mapper = {'Hyades': 'Hyades',
               'Coma Berenices Cluster':'ComaBer',
               'Pleiades':'Pleiades',
               'Praesepe': 'Praesepe',
               'alpha Per':'alphaPer',
               'IC 2391': 'IC2391',
               'IC 2602': 'IC2602',
               'Blanco 1': 'Blanco1',
               'NGC 2451A': 'NGC2451'}
cluster_info_old = querySIMBAD(name_mapper, formatGaia=True)

In [3]:
#construct a dict mapping cluster name in Table1a to its name in Simabad
cluster_info = getClusterInfo()

In [4]:
cluster_info[['cluster','Z','DM']]

cluster,Z,DM
str8,float64,float64
Blanco1,0.017,6.876
ComaBer,0.017,4.669
Hyades,0.02,3.389
IC2391,0.017,5.9079999999999995
IC2602,0.017,5.914
NGC2451,0.017,6.433
Pleiades,0.017,5.667000000000001
Praesepe,0.02,6.35
alphaPer,0.02,6.2139999999999995


In [5]:
from tabulate import tabulate

In [6]:
def coord_comp(old, new):
    old_pm_ra_cosdec = old.pm_ra_cosdec if hasattr(old,'pm_ra_cosdec') else old.pm_ra*np.cos(old.dec)
    new_pm_ra_cosdec = new.pm_ra_cosdec if hasattr(new,'pm_ra_cosdec') else new.pm_ra*np.cos(new.dec)
    
    tbl = [
        ['ra', old.ra.value, new.ra.value, (new.ra/old.ra-1)*100],
        ['dec', old.dec.value, new.dec.value, (new.dec/old.dec-1)*100],
        ['dist', old.distance.value, new.distance.value, (new.distance/old.distance-1)*100],
        ['pm_ra_cosdec', old_pm_ra_cosdec.value, new_pm_ra_cosdec.value, (new_pm_ra_cosdec/old_pm_ra_cosdec-1)*100],
        ['pmdec', old.pm_dec.value, new.pm_dec.value, (new.pm_dec/old.pm_dec-1)*100],
    ]
    
    return tabulate(tbl, headers=['Variable', 'Old', 'New', '%diff'],floatfmt=(".2f", ".2f",".2f",".2f"))


In [7]:
for cl in name_mapper:
    print(f"--------- {name_mapper[cl]} ------------")
    print(coord_comp(cluster_info_old.loc[name_mapper[cl]]['coords'], cluster_info.loc[name_mapper[cl]]['coords']))
    print()

--------- Hyades ------------
Variable         Old     New    %diff
------------  ------  ------  -------
ra             66.73   67.98     1.88
dec            15.87   17.01     7.21
dist           47.50   47.50    -0.00
pm_ra_cosdec  104.92  101.00    -3.73
pmdec         -28.00  -28.49     1.75

--------- ComaBer ------------
Variable         Old     New    %diff
------------  ------  ------  -------
ra            186.27  185.98    -0.16
dec            26.10   26.09    -0.04
dist           85.91   85.91     0.00
pm_ra_cosdec  -12.11  -12.11     0.00
pmdec          -9.00   -9.00    -0.00

--------- Pleiades ------------
Variable         Old     New    %diff
------------  ------  ------  -------
ra             56.75   56.44    -0.55
dec            24.12   23.86    -1.06
dist          135.80  135.80    -0.00
pm_ra_cosdec   20.00   20.00     0.00
pmdec         -45.55  -45.55     0.00

--------- Praesepe ------------
Variable         Old     New    %diff
------------  ------  ------  ------